In [76]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [77]:
rent_csv_file = './data/renttherunway_final_data.csv' 
# mete_csv_file = 'modcloth_final_data.csv' 

rent_data = pd.read_csv(rent_csv_file)
# mete_data = pd.read_csv(mete_csv_file)

In [78]:
def height_to_cm(height):
    if not isinstance(height, str): 
        return None 
    match = re.match(r"(\d+)' (\d+)\"", height)
    if match:
        feet = int(match.group(1)) 
        inches = int(match.group(2)) 
        return feet * 30.48 + inches * 2.54
    else:
        return None

convert height, weight. Using mean fill nan.

In [79]:
rent_data['height_cm'] = rent_data['height'].apply(height_to_cm)
rent_data['weight_kg'] = rent_data['weight'].str.replace('lbs', '').astype(float) * 0.453592

In [80]:
columns_to_fill = ['height_cm', 'weight_kg']  
for col in columns_to_fill:
    mean_value = rent_data[col].mean()  
    rent_data[col] = rent_data[col].fillna(mean_value)  

change clothing size by sort out extreme value, using 98% of the clothing size

In [81]:
threshold = rent_data['size'].quantile(0.98)
df1 = rent_data[rent_data['size'] <= threshold]

In [84]:
# def plot_combined_distributions(data_full, data_filtered):
#     fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

#     # Full data distribution
#     sns.countplot(ax=axes[0], x='size', data=data_full, order=sorted(data_full['size'].unique()))
#     axes[0].set_xlabel('Clothing Size')
#     axes[0].set_ylabel('Count')
#     axes[0].set_title('Full Distribution of Clothing Size')
#     axes[0].axvline(x=np.percentile(data_full['size'], 98), color='red', linestyle='--', label='98th Percentile')
#     axes[0].legend()
#     axes[0].tick_params(axis='x', rotation=45)

#     # Filtered data distribution
#     sns.countplot(ax=axes[1], x='size', data=data_filtered, order=sorted(data_filtered['size'].unique()))
#     axes[1].set_xlabel('Clothing Size')
#     axes[1].set_title('Filtered Distribution of Clothing Size (<= 98th Percentile)')
#     axes[1].tick_params(axis='x', rotation=45)
    
    
#     # Adjust layout
#     plt.tight_layout()
#     plt.savefig('Clothing_size.png')
    
#     plt.show()

# # Plot
# plot_combined_distributions(rent_data, df1)

drop nan on body type & add one hot encoding

In [97]:
rent_data= df1.dropna(subset=['body type'])

In [99]:
rent_data = pd.get_dummies(rent_data, columns=['body type'], prefix='body_type')

balanced the data

In [100]:
fit_counts = rent_data['fit'].value_counts()
fit_counts 

fit
fit      128856
small     23182
large     22606
Name: count, dtype: int64

In [101]:
target_size_large = fit_counts['large']


small_data_adjusted = rent_data[rent_data['fit'] == 'small'].sample(n=target_size_large, random_state=42, replace=False)
fit_data_adjusted = rent_data[rent_data['fit'] == 'fit'].sample(n=target_size_large, random_state=42, replace=False)
large_data_adjusted = rent_data[rent_data['fit'] == 'large']


balanced_data_adjusted = pd.concat([small_data_adjusted, fit_data_adjusted, large_data_adjusted])


balanced_data_adjusted = balanced_data_adjusted.sample(frac=1, random_state=42).reset_index(drop=True)

Drop useless column

In [102]:
df = balanced_data_adjusted.drop(
    columns=['bust size', 'weight', 'rented for', 'review_text', 'category', 'height', 'age', 'review_date']
)

ADD BMI feature

In [103]:
df['height_m'] = df['height_cm'] / 100
df['BMI'] = df['weight_kg'] / (df['height_m'] ** 2)
df.drop(columns=['height_m'], inplace=True)

,fit,user_id,item_id,rating,review_summary,size,height_cm,weight_kg,body_type_apple,body_type_athletic,body_type_full bust,body_type_hourglass,body_type_pear,body_type_petite,body_type_straight & narrow,BMI
0,large,773847,2587651,10.0,"Beautiful cut, color and lines! Elegant fabric...",1,154.94,47.62716,False,True,False,False,False,False,False,19.839357
1,fit,96369,597613,10.0,Want to wear this every day!,4,162.56,49.89512,False,False,False,False,False,False,True,18.881248
2,large,658008,2487984,10.0,So cozy for a rainy foggy day. My husband thin...,14,170.18,58.96696,False,False,False,True,False,False,False,20.360653
3,large,822017,2150654,6.0,kept me warm!,8,170.18,65.77084,False,False,False,True,False,False,False,22.709959
4,fit,291190,178527,10.0,Sad to send it back!,4,157.48,56.69900,False,True,False,False,False,False,False,22.862546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67813,fit,306218,2679540,10.0,Fun and Cute,8,175.26,63.50288,False,True,False,False,False,False,False,20.674157
67814,small,129868,124204,8.0,"I wore this for the my 30th birthday, a great ...",20,172.72,81.64656,False,False,False,False,True,False,False,27.368603
67815,large,352647,143094,10.0,Very fun dress!,4,162.56,56.69900,False,True,False,False,False,False,False,21.455963
67816,small,818223,877767,10.0,Black Tie Beauty!,17,167.64,68.03880,False,True,False,False,False,False,False,24.210365
